In [2]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By

import bs4
import re

import pandas as pd

## For a player

In [3]:
options = webdriver.EdgeOptions()
# options.add_argument('headless')
options.add_argument('inprivate')
driver = webdriver.Edge(options= options)

In [4]:
driver.get('https://qlstats.net/player/61491')

### Cookie screen

In [5]:
def is_cookie_screen(driver : selenium.webdriver):
    '''A function to check if the given webpage is the 'accept cookies' screen.
    Regex matches the body of the '''
    element = driver.find_element(By.TAG_NAME, value = 'body')
    cookie_screen = re.compile(r'.*(To continue using qlstats, you need to agree to the use of cookies.\nAgree).*')
    if cookie_screen.search(element.text) is None:
        return False
    return True


In [6]:
#press the button
if is_cookie_screen(driver):
    try:
        button = driver.find_element(By.TAG_NAME, 'button')
        button.click()
    except:
        "Cookies could not be accepted, please recheck"

### Scraping the player page

In [7]:
from bs4 import BeautifulSoup

In [8]:
soup = BeautifulSoup(driver.page_source, 'lxml')


In [9]:
# finding player name
player_name = soup.select_one('h2').text
print('Player name:\n{}'.format(player_name))

#for unavailable IDs:
if re.match(r"Sorry, that player wasn't found!", player_name):
    print("Player not found, can't proceed")

else:
    pass
    #TODO Scrape here

Player name:
=*=Topgun=*=


## ELSE tag here onwards

- p_tab_list
- player_info
- games_played
- stats

The flow for gametypes is as follows:
- Winrate
- K/D ratio
- Cap Ratio
- ELO
- B-ELO
- Rank
- Last Played
- Games played
- Favourite map

### RegEx Extractors

In [10]:
stat_extractor = re.compile(r"Win Rate: (.*) Kill Ratio: (.*) Cap Ratio: (.*) Rating: (.*) B-Rating: (.*) Rank: (.*) Last Played: (.*) Games Played: (.*) Favorite Map: (.*)")
winrate_extractor = re.compile(r"([0-9]+\.?[0-9]*).*\((\d+).*, ([0-9]+).*")
kdRatio_extractor = re.compile(r"(\d+\.?\d+) \((\d+).*, (\d+).*")
capRatio_extractor = re.compile(r'(.*)')
elo_extractor = re.compile(r"([0-9]+) ± ([0-9]+).*, ([0-9]+).*")
rank_extractor = re.compile(r"([0-9]+) of ([0-9]+).*")

extractors = [winrate_extractor, kdRatio_extractor, capRatio_extractor, elo_extractor, elo_extractor, rank_extractor]

In [11]:
# parse information from the p tab
def extract_player_info(soup):
    p_tab_text = soup.select_one('p').text
    p_tab_list = list(map(str.strip, p_tab_text.split("\n")))
    p_tab_list = [x for x in p_tab_list if x != '']
    return p_tab_list



def extract_player_name_details(p_tab_list):
    player_info = {'name' : player_name}
    for x in p_tab_list:
        elements = x.split(": ")
        player_info[elements[0]] = elements[1]
    return player_info



# # press the overall button on the player page
# button = driver.find_element(By.CLASS_NAME, 'tab-overall')
# button.click()

def extract_game_info(soup):
    games_played = {}

    gametypes = soup.find('ul', id = 'gbtab').text
    gametypes = [x.strip() for x in gametypes.split('\n') if x != '']

    for i in range(len(gametypes)//2):
        t, n = gametypes[i*2], gametypes[2*i+1][1:-1]
        games_played[t] = int(n)

    games_played.pop('overall')
    return games_played


def parse_gametype_stats(data:tuple):
    '''Function for parsing the stats info given a tuple of
    - winrate
    - cap ratio
    - elo
    - b-elo
    - rank
    -last played
    - games played
    - fav map
    
    Returns a dictionary containing parsed info and keys:
    winrate, cap_ratio, elo (ordered-tuple as value), b-elo (ordered-tuple as value), rank (ordered-tuple as value), last_played, num_games, fav_map'''
    ret_dict = {}
    keys = ['winrate', 'kd_ratio', 'cap_ratio', 'elo', 'b-elo', 'rank', 'last_played', 'num_games', 'fav_map']
    for i in range(6):
        try:
            ext = extractors[i]
            to_parse = data[i]
            ret_dict[keys[i]] = ext.search(to_parse).groups()
        except:
            ret_dict[keys[i]] = '-'
    return ret_dict

def parse_stats(soup, gametypes):
    '''Parse and stores stats by gametype into a dictionary.
    
    Arguments: 
    - soup: the BS4 object
    - gametypes: a list of gametypes for extracting data. This can be acquired by scraping the 'gbtab' 
    list on a player's page on QLStats.
    
    Returns: A dictionary with keys as gametypes and values as the stats associated with them.
    The stats are extracted using RegEx.'''
    stats = {}
    # parse winrate and elo info
    for gt in gametypes:
        type_stats = soup.find('div', id = 'tab-{}'.format(gt)).text
        type_stats = type_stats.split('\n')
        type_stats = [x.strip() for x in type_stats if x.strip() != '']
        stats_as_string = " ".join(type_stats)

        res = stat_extractor.search(stats_as_string)
        #there is winrate, kill ratio, cap ratio, rating, b-rating, rank, last played games played favorite map
        stat_dict = parse_gametype_stats(res.groups())
        stats[gt] = stat_dict

    return stats





p_tab_list = extract_player_info(soup)
player_info = extract_player_name_details(p_tab_list)
games_played = extract_game_info(soup)
stats = parse_stats(soup, games_played.keys())
print(p_tab_list)
print(player_info)
print(games_played)
print(stats)

['Region: Asia', 'Player ID: 61491', 'Steam ID: 76561197965738638', 'Joined: 2016-02-11   19:57:19', 'Status: active']
{'name': '=*=Topgun=*=', 'Region': 'Asia', 'Player ID': '61491', 'Steam ID': '76561197965738638', 'Joined': '2016-02-11   19:57:19', 'Status': 'active'}
{'ca': 15098, 'duel': 582, 'ffa': 391, 'dom': 33, 'ctf': 14, 'rr': 6, 'tdm': 5, 'harvester': 4, 'ft': 3, 'ad': 3, 'race': 2}
{'ca': {'winrate': ('59.85', '9036', '6062'), 'kd_ratio': ('1.3', '181429', '139874'), 'cap_ratio': ('-',), 'elo': ('1341', '33', '10385'), 'b-elo': ('1216', '46', '479'), 'rank': ('9', '1515')}, 'duel': {'winrate': ('59.11', '344', '238'), 'kd_ratio': ('1.14', '6288', '5521'), 'cap_ratio': ('-',), 'elo': ('1310', '249', '343'), 'b-elo': ('738', '290', '1'), 'rank': '-'}, 'ffa': {'winrate': ('53.71', '210', '181'), 'kd_ratio': ('1.56', '16273', '10462'), 'cap_ratio': ('-',), 'elo': ('1825', '319', '18'), 'b-elo': ('1538', '93', '247'), 'rank': '-'}, 'dom': {'winrate': ('54.55', '18', '15'), 'kd_r

Overall tab just has the latest/maximal values from the other tabs.

In [12]:
soup.findAll('table', class_= 'table table-hover table-condensed')

[<table class="table table-hover table-condensed">
 <thead>
 <tr>
 <th>#</th>
 <th></th>
 <th>Played</th>
 <th>Type</th>
 <th>Server</th>
 <th>Map</th>
 <th>Result</th>
 <th>Opponent</th>
 <th>Rating</th>
 <th title="Rating ± Uncertainty">Old Glicko</th>
 <th title="Rating / Uncertainty">Glicko Change</th>
 </tr>
 </thead>
 <tbody><tr><td>1</td>
 <td class="tdcenter"><a class="btn btn-primary btn-small" href="/game/8565965" title="View detailed information about this game">view</a></td>
 <td>2023-02-18 22:10</td>
 <td><img alt="ca" height="24" src="/static/images/icons/24x24/ca.png" title="Clan Arena" width="24"/> ca</td>
 <td><a href="/server/5935" title="Go to the detail page for this server">(India+Singapore+UAE) Topgun's CA Server w/crouchslide</a></td>
 <td><a href="/map/44" title="Go to the detail page for this map">almostlost</a></td>
 <td class="elodown">10:8 (#4)</td>
 <td><a href="/player/-1"><span class="ql7">Anonymous 1</span></a></td>
 <td>945 ± 53</td>
 <td>1333 ± 33</td>

## Scrape match data

In [13]:
#click on 'more' button
try:
    btn = driver.find_element(By.LINK_TEXT, 'More...')
    btn.click()

    btn = driver.find_element(By.CSS_SELECTOR, '[alt="overall"]')
    btn.click()
except:
    print('Could not click the button')

Could not click the button


In [14]:
pd.read_html(driver.current_url, attrs = {'class': 'table table-hover table-condensed'})

[   Unnamed: 0              Played Type  \
 0        view      11 minutes ago   ca   
 1        view      11 minutes ago   ca   
 2        view      25 minutes ago   ca   
 3        view      41 minutes ago   ca   
 4        view    about 1 hour ago   ca   
 5        view    about 1 hour ago   ca   
 6        view    about 1 hour ago   ca   
 7        view  about 21 hours ago   ca   
 8        view  about 21 hours ago   ca   
 9        view  about 21 hours ago   ca   
 10       view  about 22 hours ago   ca   
 11       view  about 22 hours ago   ca   
 12       view  about 22 hours ago   ca   
 13       view  about 22 hours ago   ca   
 14       view  about 23 hours ago   ca   
 15       view  about 23 hours ago   ca   
 16       view  about 23 hours ago   ca   
 17       view  about 24 hours ago   ca   
 18       view  about 24 hours ago   ca   
 19       view           1 day ago   ca   
 
                                                Server             Map  \
 0   (India+Singapore

### Shut down the browser

In [15]:
# driver.close()